In [2]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv('/content/Creditcard_data.csv')
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,1
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
print(data.shape)
data.Class.value_counts()

(772, 31)


0    763
1      9
Name: Class, dtype: int64

In [5]:
data_0 = data[data['Class'] == 0]
data_1 = data[data['Class'] == 1]

In [6]:
x = data.iloc[:, :-1]
y = data.iloc[:, -1]
class1 = np.where(y==1)
class0 = np.where(y==0)
len(class0[0]), len(class1[0])

(763, 9)

In [7]:
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
RandomOver = RandomOverSampler(random_state=42)
x_over, y_over = RandomOver.fit_resample(x, y)

print('Original dataset shape', Counter(y))
print('Resample dataset shape', Counter(y_over))

Original dataset shape Counter({0: 763, 1: 9})
Resample dataset shape Counter({0: 763, 1: 763})


In [8]:
df= pd.concat([x_over, y_over], axis= 1)
df.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

In [9]:
len(data_0)

763

In [10]:
import math
N = len(data_0)
p = 0.5
e = 0.05
Zscore = 1.96

srs_ssize = math.ceil((Zscore**2 * p * (1-p)) / (e**2))
print("Sample Size for Simple Random Sampling: ", srs_ssize)

strata_size = y_over.value_counts()
total = strata_size.sum()
strata_size = data["Class"].value_counts()
total = strata_size.sum()
strata_ssize = math.ceil((Zscore**2 * p * (1-p) * total) / ((total - 1) * e**2 + Zscore**2 * p * (1-p)))
print("Sample Size for Stratified Random Sampling: ",strata_ssize)

clusters = 8
cluster_ssize = math.ceil((Zscore**2 * p * (1-p) * clusters) / ((clusters - 1) * e**2 + Zscore**2 * p * (1-p)))
print("Sample Size for Cluster Sampling: ", cluster_ssize)

sys_ssize = math.ceil(math.sqrt(N))
print("Sample Size for Systematic Sampling: ", sys_ssize)


Sample Size for Simple Random Sampling:  385
Sample Size for Stratified Random Sampling:  257
Sample Size for Cluster Sampling:  8
Sample Size for Systematic Sampling:  28


In [11]:
srs_dataSample= pd.DataFrame(df.sample(n=srs_ssize))
srs_testData = df.merge(srs_dataSample, how='outer', indicator=True).query('_merge == "left_only"').drop('_merge', 1)


<ipython-input-11-100d95914841>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  srs_testData = df.merge(srs_dataSample, how='outer', indicator=True).query('_merge == "left_only"').drop('_merge', 1)


In [12]:
sys_dataSample= pd.DataFrame(df.iloc[::sys_ssize])
sys_testData= df.merge(sys_dataSample,how = 'outer',indicator=True).query('_merge == "left_only"').drop('_merge', 1)


<ipython-input-12-41d0be627352>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  sys_testData= df.merge(sys_dataSample,how = 'outer',indicator=True).query('_merge == "left_only"').drop('_merge', 1)


In [13]:
from sklearn.model_selection import StratifiedShuffleSplit
Y = "Class"
P = [x for x in data.columns if x != Y]

StraSplit = StratifiedShuffleSplit(n_splits=1,  train_size=strata_ssize, random_state=42)
for train_index, _ in StraSplit.split(data[P], data[Y]):
    strata_dataSample = pd.DataFrame(data.iloc[train_index])

In [14]:
strata_testData = df.merge(strata_dataSample,how = 'outer',indicator=True).query('_merge == "left_only"').drop('_merge', 1)


<ipython-input-14-631fc5d4e9f1>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  strata_testData = df.merge(strata_dataSample,how = 'outer',indicator=True).query('_merge == "left_only"').drop('_merge', 1)


In [15]:
import random
cluster_size= math.ceil(N / clusters)
clusters_ = [data.iloc[i:i+cluster_size] for i in range(0, len(data), cluster_size)]

cluster_dataSample= (random.choice(clusters_))

cluster_testData= df.merge(cluster_dataSample, how = 'outer' ,indicator=True).query('_merge == "left_only"').drop('_merge', 1)


<ipython-input-15-6d695ae4fd98>:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  cluster_testData= df.merge(cluster_dataSample, how = 'outer' ,indicator=True).query('_merge == "left_only"').drop('_merge', 1)


In [16]:
BalancedData = pd.concat([x_over, y_over], axis=1)

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

Model = { 'M1': LogisticRegression(max_iter=200),
          'M2': DecisionTreeClassifier(criterion='gini',max_depth=5,random_state=33),
          'M3': RandomForestClassifier(criterion = 'gini',n_estimators=200,max_depth=5,random_state=33, n_jobs=-1),
          'M4': GaussianNB(),
          'M5': KNeighborsClassifier(n_neighbors=3)}

# define the samples
Sample = {"Sample 1": [srs_dataSample, srs_testData],
            "Sample 2": [sys_dataSample, sys_testData],
            "Sample 3": [strata_dataSample, strata_testData],
            "Sample 4": [cluster_dataSample, cluster_testData],
            "Sample 5": 'None'}

In [28]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

score = pd.DataFrame(columns = [sample_name for sample_name, _ in Sample.items()], index= [model_name for model_name, _ in Model.items()])

for sample_name, sample in Sample.items():
    for model_name, model in Model.items():
        if sample_name == 'Sample 5':
            X_train, X_test, y_train, y_test = train_test_split(BalancedData.drop('Class', axis=1), BalancedData['Class'], test_size=0.2)
        else:
            X_train, y_train = sample[0].drop('Class', axis=1), sample[0]['Class']
            X_test, y_test = sample[1].drop('Class', axis=1), sample[1]['Class']

        model.fit(X_train, y_train)
        y_pred =model.predict(X_test)
        
        metric = metrics.accuracy_score(y_test, y_pred)
        score.loc[model_name,sample_name] = metric

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

In [65]:
def get_classification_metric(testy, probs):
    from sklearn.metrics import precision_recall_curve
    precision, recall, thresholds = precision_recall_curve(testy, probs[:,1])
    # convert to f score
    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    return fscore[ix]

In [30]:
score

,Sample 1,Sample 2,Sample 3,Sample 4,Sample 5
M1,0.858657,0.782787,0.49901,0.522601,0.944444
M2,0.959364,0.85929,0.488119,0.509913,0.980392
M3,0.987633,0.969945,0.5,0.530531,0.993464
M4,0.939929,0.662568,0.49802,0.530531,0.75817
M5,0.932862,0.693989,0.5,0.530531,0.993464
